In [3]:
import numpy as np
import sympy as sm
import sympy.physics.mechanics as me
from sympy.physics.vector import init_vprinting, vlatex
from pydy.codegen.ode_function_generators import generate_ode_function
from scipy.integrate import odeint
from pydy.viz.shapes import Cylinder, Sphere
import pydy.viz
pydy.viz.shapes.__all__
from pydy.viz.visualization_frame import VisualizationFrame
from pydy.viz.scene import Scene

viz_switch = True

# Kinematics

**Reference Frames:** Declare global and body fixed reference frames

In [4]:
world_frame = me.ReferenceFrame('W')
pelvis_frame = me.ReferenceFrame('P')
torso_frame = me.ReferenceFrame('T')
right_thigh_frame = me.ReferenceFrame('RT')
left_thigh_frame = me.ReferenceFrame('LT')
right_shank_frame = me.ReferenceFrame('RS')
left_shank_frame = me.ReferenceFrame('LS')
right_foot_frame = me.ReferenceFrame('RF')
left_foot_frame = me.ReferenceFrame('LF')

**Coordinates:** Declare coordinates for every degree of freedom

In [5]:
qPx,qPy,qPz,qL,qRH,qLH,qRK,qLK,qRA,qLA = me.dynamicsymbols('qPx qPy qPz qL qRH qLH qRK qLK qRA qLA')

**Orient Frames:** Orient each of the frame using the declared coordinates

In [6]:
pelvis_frame.orient(world_frame,'Axis',(qPz,world_frame.z))
torso_frame.orient(pelvis_frame,'Axis',(qL,pelvis_frame.z))
right_thigh_frame.orient(pelvis_frame,'Axis',(qRH,pelvis_frame.z))
left_thigh_frame.orient(pelvis_frame,'Axis',(qLH,pelvis_frame.z))
right_shank_frame.orient(right_thigh_frame,'Axis',(qRK,right_thigh_frame.z))
left_shank_frame.orient(left_thigh_frame,'Axis',(qLK,left_thigh_frame.z))
right_foot_frame.orient(right_shank_frame,'Axis',(qRA,right_shank_frame.z))
left_foot_frame.orient(left_shank_frame,'Axis',(qLA,left_shank_frame.z))

**Joints:** Set the position of each joint 

In [7]:
#Declare the necessary length of each segment
pelvis_length, torso_length, thigh_length, shank_length, foot_length = sm.symbols('L_pelvis L_torso L_thigh L_shank L_foot')
#Declare the width of the foot
foot_width = sm.symbols('W_foot')
#Height of the CoM - relative to the bottom end of the segment
pelvis_com_length, torso_com_length, thigh_com_length, shank_com_length, foot_com_length = sm.symbols('c_pelvis c_torso c_thigh c_shank c_foot')
#Forward position of the foot CoM
foot_com_length_x = sm.symbols('cx_foot')

#Origin
O = me.Point('O')
#Root - i.e. center of mass of the pelvis
root = me.Point('root')
root.set_pos(O,qPx*world_frame.x + qPy*world_frame.y)
#Lumbar joint
lumbar = me.Point('L')
lumbar.set_pos(root,(pelvis_length-pelvis_com_length)*pelvis_frame.y)
#Right hip
right_hip = me.Point('RH')
right_hip.set_pos(root,-pelvis_com_length*pelvis_frame.y)
#Left hip
left_hip = me.Point('LH')
left_hip.set_pos(root,-pelvis_com_length*pelvis_frame.y)
#Right Knee
right_knee = me.Point('RK')
right_knee.set_pos(right_hip,-thigh_length*right_thigh_frame.y)
#Left Knee 
left_knee = me.Point('LK')
left_knee.set_pos(left_hip,-thigh_length*left_thigh_frame.y)
#Right Ankle
right_ankle = me.Point('RA')
right_ankle.set_pos(right_knee,-shank_length*right_shank_frame.y)
#Left Ankle
left_ankle = me.Point('LA')
left_ankle.set_pos(left_knee,-shank_length*left_shank_frame.y)

**Center of Mass:** Set the position of the center of mass of each segment

In [8]:
#Pelvis
pelvis_mass_center = me.Point('P_CoM')
pelvis_mass_center.set_pos(root,0)
#Torso
torso_mass_center = me.Point('T_CoM')
torso_mass_center.set_pos(lumbar,torso_com_length*torso_frame.y)
#Right Thigh
right_thigh_mass_center = me.Point('RT_CoM')
right_thigh_mass_center.set_pos(right_hip,-1*(thigh_length-thigh_com_length)*right_thigh_frame.y)
#Left Thigh
left_thigh_mass_center = me.Point('LT_CoM')
left_thigh_mass_center.set_pos(left_hip,-1*(thigh_length-thigh_com_length)*left_thigh_frame.y)
#Right Shank
right_shank_mass_center = me.Point('RS_CoM')
right_shank_mass_center.set_pos(right_knee,-1*(shank_length-shank_com_length)*right_shank_frame.y)
#Left Shank
left_shank_mass_center = me.Point('LS_CoM')
left_shank_mass_center.set_pos(left_knee,-1*(shank_length-shank_com_length)*left_shank_frame.y)
#Right Foot
right_foot_mass_center = me.Point('RF_CoM')
right_foot_mass_center.set_pos(right_ankle,-1*(foot_length - foot_com_length)*right_foot_frame.y + foot_com_length_x*right_foot_frame.x)
#Left Foot
left_foot_mass_center = me.Point('LF_CoM')
left_foot_mass_center.set_pos(left_ankle,-1*(foot_length-foot_com_length)*left_foot_frame.y + foot_com_length_x*left_foot_frame.x)

**Ground Reaction Force Sites:** Add points for the contact between the foot and the ground

In [9]:
#Right Heel
right_heel = me.Point('RH')
right_heel.set_pos(right_ankle,-foot_length*right_foot_frame.y)
#Right Toe
right_toe = me.Point('RT')
right_toe.set_pos(right_ankle,-foot_length*right_foot_frame.y + foot_width*right_foot_frame.x)
#Left Heel
left_heel = me.Point('LH')
left_heel.set_pos(left_ankle,-foot_length*left_foot_frame.y)
#Right Toe
left_toe = me.Point('LT')
left_toe.set_pos(left_ankle,-foot_length*left_foot_frame.y + foot_width*left_foot_frame.x)


**Kinematic Differential Equations:** Declare generalized speeds and relate them to each of the generalized coordinates to determine the kinematical differential equations

In [10]:
vPx,vPy,vPz,vL,vRH,vLH,vRK,vLK,vRA,vLA = me.dynamicsymbols('vPx vPy vPz vL vRH vLH vRK vLK vRA vLA')

kinematical_differential_equations = [vPx - qPx.diff(),
                                      vPy - qPy.diff(),
                                      vPz - qPz.diff(),
                                      vL - qL.diff(),
                                      vRH - qRH.diff(),
                                      vLH - qLH.diff(),
                                      vRK - qRK.diff(),
                                      vLK - qLK.diff(),
                                      vRA - qRA.diff(),
                                      vLA - qLA.diff()]

**Angular Velocities:** Use the generalized speeds to define the angular velocity of each frame

In [11]:
#Pelvis
pelvis_frame.set_ang_vel(world_frame,vPz*world_frame.z)
pelvis_frame.ang_vel_in(world_frame)
#Torso
torso_frame.set_ang_vel(pelvis_frame,vL*pelvis_frame.z)
torso_frame.ang_vel_in(world_frame)
#Right Thigh
right_thigh_frame.set_ang_vel(pelvis_frame,vRH*pelvis_frame.z)
right_thigh_frame.ang_vel_in(world_frame)
#Left Thigh
left_thigh_frame.set_ang_vel(pelvis_frame,vLH*pelvis_frame.z)
left_thigh_frame.ang_vel_in(world_frame)
#Right Shank
right_shank_frame.set_ang_vel(right_thigh_frame,vRK*right_thigh_frame.z)
right_shank_frame.ang_vel_in(world_frame)
#Left Shank
left_shank_frame.set_ang_vel(left_thigh_frame,vLK*left_thigh_frame.z)
left_shank_frame.ang_vel_in(world_frame)
#Right Foot
right_foot_frame.set_ang_vel(right_shank_frame,vRA*right_shank_frame.z)
right_foot_frame.ang_vel_in(world_frame)
#Left Foot
left_foot_frame.set_ang_vel(left_shank_frame,vLA*left_shank_frame.z)
left_foot_frame.ang_vel_in(world_frame)

vPz(t)*W.z + vLH(t)*P.z + vLK(t)*LT.z + vLA(t)*LS.z

**Linear Velocities:** Define the linear velocity of each CoM   

In [12]:
O.set_vel(world_frame,0)

#Pelvis
root.set_vel(world_frame,vPx*world_frame.x + vPy*world_frame.y)
root.vel(world_frame)
pelvis_mass_center.set_vel(world_frame,vPx*world_frame.x + vPy*world_frame.y)
pelvis_mass_center.vel(world_frame)
#Lumbar
lumbar.v2pt_theory(root,world_frame,pelvis_frame)
lumbar.vel(world_frame)
#Torso
torso_mass_center.v2pt_theory(lumbar,world_frame,torso_frame)
torso_mass_center.vel(world_frame)
#Right Hip
right_hip.v2pt_theory(root,world_frame,pelvis_frame)
right_hip.vel(world_frame)
#Left Hip
left_hip.v2pt_theory(root,world_frame,pelvis_frame)
left_hip.vel(world_frame)
#Right thigh
right_thigh_mass_center.v2pt_theory(right_hip,world_frame,right_thigh_frame)
right_thigh_mass_center.vel(world_frame)
#Left thigh
left_thigh_mass_center.v2pt_theory(left_hip,world_frame,left_thigh_frame)
left_thigh_mass_center.vel(world_frame)
#Right Knee
right_knee.v2pt_theory(right_hip,world_frame,right_thigh_frame)
right_knee.vel(world_frame)
#Left Knee
left_knee.v2pt_theory(left_hip,world_frame,left_thigh_frame)
left_knee.vel(world_frame)
#Right Shank
right_shank_mass_center.v2pt_theory(right_knee,world_frame,right_shank_frame)
right_shank_mass_center.vel(world_frame)
#Left Shank
left_shank_mass_center.v2pt_theory(left_knee,world_frame,left_shank_frame)
left_shank_mass_center.vel(world_frame)
#Right Ankle
right_ankle.v2pt_theory(right_knee,world_frame,right_shank_frame)
right_ankle.vel(world_frame)
#Left Ankle
left_ankle.v2pt_theory(left_knee,world_frame,left_shank_frame)
left_ankle.vel(world_frame)
#Right foot
right_foot_mass_center.v2pt_theory(right_ankle,world_frame,right_foot_frame)
right_foot_mass_center.vel(world_frame)
#Left foot
left_foot_mass_center.v2pt_theory(left_ankle,world_frame,left_foot_frame)
left_foot_mass_center.vel(world_frame)
#Right Heel
right_heel.v2pt_theory(right_ankle,world_frame,right_foot_frame)
right_heel.vel(world_frame)
#Right Toe
right_toe.v2pt_theory(right_ankle,world_frame,right_foot_frame)
right_toe.vel(world_frame)
#Left Heel
left_heel.v2pt_theory(left_ankle,world_frame,left_foot_frame)
left_heel.vel(world_frame)
#Right Toe
left_toe.v2pt_theory(left_ankle,world_frame,left_foot_frame)
left_toe.vel(world_frame)

vPx(t)*W.x + vPy(t)*W.y + c_pelvis*vPz(t)*P.x + L_thigh*(vLH(t) + vPz(t))*LT.x + L_shank*(vLH(t) + vLK(t) + vPz(t))*LS.x + L_foot*(vLA(t) + vLH(t) + vLK(t) + vPz(t))*LF.x + W_foot*(vLA(t) + vLH(t) + vLK(t) + vPz(t))*LF.y

# Inertia  

**Mass:** Define the mass of each segment  

In [13]:
pelvis_mass, torso_mass, thigh_mass, shank_mass, foot_mass = sm.symbols('m_pelvis m_torso m_thigh m_shank m_foot')

**Moment of Inertia:** Define the inertia tensor of each segment    

In [14]:
pelvis_inertia, torso_inertia, thigh_inertia, shank_inertia, foot_inertia = sm.symbols('I_pelvis I_torso I_thigh I_shank I_foot')

#Pelvis 
pelvis_dyadic = me.inertia(pelvis_frame,0,0,pelvis_inertia)
pelvis_central_inertia = (pelvis_dyadic,pelvis_mass_center)
#Torso
torso_dyadic = me.inertia(torso_frame,0,0,torso_inertia)
torso_central_inertia = (torso_dyadic,torso_mass_center)
#Right Thigh
right_thigh_dyadic = me.inertia(right_thigh_frame,0,0,thigh_inertia)
right_thigh_central_inertia = (right_thigh_dyadic,right_thigh_mass_center)
#Left Thigh
left_thigh_dyadic = me.inertia(left_thigh_frame,0,0,thigh_inertia)
left_thigh_central_inertia = (left_thigh_dyadic,left_thigh_mass_center)
#Right Shank
right_shank_dyadic = me.inertia(right_shank_frame,0,0,shank_inertia)
right_shank_central_inertia = (right_shank_dyadic,right_shank_mass_center)
#Left Shank
left_shank_dyadic = me.inertia(left_shank_frame,0,0,shank_inertia)
left_shank_central_inertia = (left_shank_dyadic,left_shank_mass_center)
#Right Foot
right_foot_dyadic = me.inertia(right_foot_frame,0,0,foot_inertia)
right_foot_central_inertia = (right_foot_dyadic,right_foot_mass_center)
#Left foot
left_foot_dyadic = me.inertia(left_foot_frame,0,0,foot_inertia)
left_foot_central_inertia = (left_foot_dyadic,left_foot_mass_center)

**Rigid Bodies:** Define rigid bodies for each of the segments

In [15]:
pelvis = me.RigidBody('Pelvis',pelvis_mass_center,pelvis_frame,
                      pelvis_mass,pelvis_central_inertia)

torso = me.RigidBody('Torso',torso_mass_center,torso_frame,
                     torso_mass,torso_central_inertia)

right_thigh = me.RigidBody('Right Thigh',right_thigh_mass_center,right_thigh_frame,
                           thigh_mass,right_foot_central_inertia)

left_thigh = me.RigidBody('Left Thigh',left_thigh_mass_center,left_thigh_frame,
                          thigh_mass,left_thigh_central_inertia)

right_shank = me.RigidBody('Right Shank',right_shank_mass_center,right_shank_frame,
                           shank_mass,right_shank_central_inertia)

left_shank = me.RigidBody('Left Shank',left_foot_mass_center,left_shank_frame,
                          shank_mass,left_shank_central_inertia)

right_foot = me.RigidBody('Right Foot',right_foot_mass_center,right_foot_frame,
                          foot_mass,right_foot_central_inertia)

left_foot = me.RigidBody('Left Foot', left_foot_mass_center,left_foot_frame,
                         foot_mass,left_foot_central_inertia)

# Kinetics

**Gravity:** Define the gravitational force for each segment            

In [16]:
g = sm.symbols('g')

#Pelvis
pelvis_grav_force_vector = -pelvis_mass*g*world_frame.y
pelvis_grav_force = (pelvis_mass_center,pelvis_grav_force_vector)

#Torso
torso_grav_force_vector = -torso_mass*g*world_frame.y
torso_grav_force = (torso_mass_center, torso_grav_force_vector)

#Right Thigh
right_thigh_grav_force_vector = -thigh_mass*g*world_frame.y
right_thigh_grav_force = (right_thigh_mass_center,right_thigh_grav_force_vector)

#Left Thigh
left_thigh_grav_force_vector = -thigh_mass*g*world_frame.y
left_thigh_grav_force = (left_thigh_mass_center,left_thigh_grav_force_vector)

#Right Shank
right_shank_grav_force_vector = -shank_mass*g*world_frame.y
right_shank_grav_force = (right_shank_mass_center,right_shank_grav_force_vector)

#Left Shank
left_shank_grav_force_vector = -shank_mass*g*world_frame.y
left_shank_grav_force = (left_shank_mass_center,left_shank_grav_force_vector)

#Right Foot
right_foot_grav_force_vector = -foot_mass*g*world_frame.y
right_foot_grav_force = (right_foot_mass_center,right_foot_grav_force_vector)

#Left Foot
left_foot_grav_force_vector = -foot_mass*g*world_frame.y
left_foot_grav_force = (left_foot_mass_center,left_foot_grav_force_vector)

**Joint Torque:** Define the torque acting at each of the joints

In [17]:
lumbar_torque = me.dynamicsymbols('T_lumbar')
right_hip_torque = me.dynamicsymbols('T_righhip')
left_hip_torque = me.dynamicsymbols('T_lefthip')
right_knee_torque = me.dynamicsymbols('T_rightknee')
left_knee_torque = me.dynamicsymbols('T_leftknee')
right_ankle_torque = me.dynamicsymbols('T_rightankle')
left_ankle_torque = me.dynamicsymbols('T_leftankle')

#Pelvis
pelvis_torque_vector = right_hip_torque*world_frame.z + left_hip_torque*world_frame.z - lumbar_torque*world_frame.z
pelvis_torque = (pelvis_frame,pelvis_torque_vector)
#Torso
torso_torque_vector = lumbar_torque*world_frame.z
torso_torque = (torso_frame,torso_torque_vector)
#Right Thigh
right_thigh_torque_vector = right_knee_torque*world_frame.z - right_hip_torque*world_frame.z
right_thigh_torque = (right_thigh_frame,right_thigh_torque_vector)
#Left Thigh
left_thigh_torque_vector = left_knee_torque*world_frame.z - left_hip_torque*world_frame.z
left_thigh_torque = (left_thigh_frame,left_thigh_torque_vector)
#Right Shank
right_shank_torque_vector = right_ankle_torque*world_frame.z - right_knee_torque*world_frame.z
right_shank_torque = (right_shank_frame,right_shank_torque_vector)
#Left Shank
left_shank_torque_vector = left_ankle_torque*world_frame.z - left_knee_torque*world_frame.z
left_shank_torque = (left_shank_frame,left_shank_torque_vector)
#Right Foot
right_foot_torque_vector = -right_ankle_torque*world_frame.z
right_foot_torque = (right_foot_frame,right_foot_torque_vector)
#Left Foot
left_foot_torque_vector = -left_ankle_torque*world_frame.z
left_foot_torque = (left_foot_frame,left_foot_torque_vector)

**Ground Reaction Force:** Add ground reaction force to the contact points

In [18]:
stiffness, damping, friction, friction_scaling, sled_velocity = sm.symbols('k, c, mu, vs, v_sled') #Characteristic of contact
OffSet = pelvis_com_length + thigh_length + shank_length + foot_length

#Right Heel
y_location = right_heel.pos_from(O).dot(world_frame.y) + OffSet
penetration = ((sm.Abs(y_location)) - y_location)/2
velocity = right_heel.vel(world_frame) - sled_velocity*world_frame.x

vertical_force = (stiffness*penetration**3 - y_location)*\
                (1 - damping*velocity.dot(world_frame.y))

friction_force = -friction*vertical_force*\
                ((2 / (1 + sm.exp(-velocity.dot(world_frame.x) /
                friction_scaling))) - 1)

F_point = 0*world_frame.x + 0*world_frame.y

right_heel_contact_force = (right_heel,F_point)


# Equations of Motion

**Kane's Method:** Use Kane's Method to formulate the equations of motion

In [19]:
coordinates = [qPx,qPy,qPz,qL,qRH,qLH,qRK,qLK,qRA,qLA]
speeds = [vPx,vPy,vPz,vL,vRH,vLH,vRK,vLK,vRA,vLA]

kane = me.KanesMethod(world_frame,coordinates,speeds,kinematical_differential_equations)

load = [pelvis_grav_force,
         torso_grav_force,
         right_thigh_grav_force,
         left_thigh_grav_force,
         right_shank_grav_force,
         left_shank_grav_force,
         right_foot_grav_force,
         left_foot_grav_force,
         pelvis_torque,
         torso_torque,
         right_thigh_torque,
         left_thigh_torque,
         right_shank_torque,
         left_shank_torque,
         right_foot_torque,
         left_foot_torque,
         right_heel_contact_force]

body = [pelvis,
          torso,
          right_thigh,
          left_thigh,
          right_shank,
          left_shank,
          right_foot,
          left_foot]

fr, frstar = kane.kanes_equations(bodies=body,loads=load)
mass_matrix = kane.mass_matrix_full
forcing_vector = kane.forcing_full

# Simulate

**Variables:** Declare the variables for the simulation

In [20]:
constants = [pelvis_length,
             torso_length,
             thigh_length,
             shank_length,
             foot_length,
             foot_width,
             pelvis_com_length,
             torso_com_length,
             thigh_com_length,
             shank_com_length,
             foot_com_length,
             foot_com_length_x,
             pelvis_mass,
             torso_mass,
             thigh_mass,
             shank_mass,
             foot_mass,
             pelvis_inertia,
             torso_inertia,
             thigh_inertia,
             shank_inertia,
             foot_inertia,
             g,
             stiffness,
             damping,
             friction,
             friction_scaling,
             sled_velocity]


coordinates = [qPx,qPy,qPz,qL,qRH,qLH,qRK,qLK,qRA,qLA]
speeds = [vPx,vPy,vPz,vL,vRH,vLH,vRK,vLK,vRA,vLA]

specified = [lumbar_torque,right_hip_torque,left_hip_torque,
             right_knee_torque,left_knee_torque,
             right_ankle_torque, left_ankle_torque]

**Generate Numerical ODE Function:** Use PyDy to generate the ODEs for odeint to integrate

In [21]:
right_hand_side = generate_ode_function(forcing_vector,
                                        coordinates,speeds,
                                        constants,
                                        mass_matrix = mass_matrix,
                                        specifieds = specified,
                                        generator = 'cython')

help(right_hand_side)

Help on function rhs in module pydy.codegen.ode_function_generators:

rhs(*args)
    Returns the derivatives of the states, i.e. numerically evaluates the right
    hand side of the first order differential equation.

    x' = f(x, t, r, p)

    Parameters
    x : ndarray, shape(20,)
        The state vector is ordered as such:
            - qPx(t)
            - qPy(t)
            - qPz(t)
            - qL(t)
            - qRH(t)
            - qLH(t)
            - qRK(t)
            - qLK(t)
            - qRA(t)
            - qLA(t)
            - vPx(t)
            - vPy(t)
            - vPz(t)
            - vL(t)
            - vRH(t)
            - vLH(t)
            - vRK(t)
            - vLK(t)
            - vRA(t)
            - vLA(t)
    t : float
        The current time.
    r : dictionary; ndarray, shape(7,); function

        There are three options for this argument. (1) is more flexible but
        (2) and (3) are much more efficient.

        (1) A dictionary that maps the s

**Function Inputs:** Initial conditions, parameter values, and time array

In [22]:
#Initial Conditions
x0 = np.zeros(20)

#Constants
numerical_constants = np.array([0.1, #Pevlis length [m]
                                1.0, #torso length [m]
                                0.6, #thigh length [m]
                                0.5, #shank length [m]
                                0.1, #foot length
                                0.3, #Foot width
                                0.05, #Pelvis com length [m]
                                0.5, #torso com length [m]
                                0.3, #thigh com length 
                                0.25, #shank com length 
                                0.05, #foot com length
                                0.15, #foot com length x
                                10.0, #pelvis mass
                                30.0, #torso mass 
                                15.0, #Thigh mass
                                10.0, #shank mass
                                5.0, #foot mass
                                3.0, #Pelvis inertia
                                8.0, #torso inertia
                                2.0, #thigh inertia
                                1.0, #shank inertia
                                0.5, #Foot inertia
                                9.81, #gravity
                                5.0e+7, #stiffness 
                                0.85, #damping 
                                1.0, #friction 
                                0.01, #friction scaling 
                                0.0]) #sled velocity

numerical_specified = np.zeros(7)

args = {'constants': numerical_constants,
        'specified': numerical_specified}

frame_per_sec = 60.0
final_time = 10.0

t = np.linspace(0.0,final_time,int(final_time*frame_per_sec))

**Integrate the ODE:** Use odeint to solve the ODEs

In [23]:
y = odeint(right_hand_side,x0,t,args=(numerical_specified,numerical_constants))

C:\Users\anbes\AppData\Local\Temp\ipykernel_14860\827528399.py:1: ODEintWarning: Excess work done on this call (perhaps wrong Dfun type). Run with full_output = 1 to get quantitative information.
  y = odeint(right_hand_side,x0,t,args=(numerical_specified,numerical_constants))


# Visualization

In [24]:
contact_shape = Sphere(color='red',radius=0.01)

right_heel_viz_frame = VisualizationFrame(world_frame,right_heel,contact_shape)
right_toe_viz_frame = VisualizationFrame(world_frame,right_toe,contact_shape)
left_heel_viz_frame = VisualizationFrame(world_frame,left_heel,contact_shape)
left_toe_viz_frame = VisualizationFrame(world_frame,left_toe,contact_shape)

ankle_shape = Sphere(color = 'black',radius = 0.05)
knee_shape = Sphere(color = 'black',radius = 0.05)
hip_shape = Sphere(color = 'black',radius = 0.05)
lumbar_shape = Sphere(color = 'black',radius = 0.05)
head_shape = Sphere(color = 'black',radius = 0.1)

right_ankle_viz_frame = VisualizationFrame(world_frame,right_ankle,ankle_shape)
left_ankle_viz_frame = VisualizationFrame(world_frame,left_ankle,ankle_shape)
right_knee_viz_frame = VisualizationFrame(world_frame,right_knee,knee_shape)
left_knee_viz_frame = VisualizationFrame(world_frame,left_knee,knee_shape)
right_hip_viz_frame = VisualizationFrame(world_frame,right_knee,hip_shape)
left_hip_viz_frame = VisualizationFrame(world_frame,left_hip,hip_shape)
lumbar_viz_frame = VisualizationFrame(world_frame,lumbar,lumbar_shape)

head = me.Point('N')
head.set_pos(lumbar,torso_length*torso_frame.y)
head_viz_frame = VisualizationFrame(world_frame,head,head_shape)

right_foot_center = me.Point('RF_c')
left_foot_center = me.Point('LF_c')
right_shank_center = me.Point('RS_c')
left_shank_center = me.Point('LS_c')
right_thigh_center = me.Point('RT_c')
left_thigh_center = me.Point('LT_c')
pelvis_center = me.Point('P_c')
torso_center = me.Point('T_c')

right_foot_center.set_pos(right_ankle,(foot_width/2)*right_foot_frame.x - (foot_length/2)*right_foot_frame.y)
left_foot_center.set_pos(left_ankle,(foot_width/2)*left_foot_frame.x - (foot_length/2)*left_foot_frame.y)
right_shank_center.set_pos(right_ankle,(shank_length/2)*right_shank_frame.y)
left_shank_center.set_pos(left_ankle,(shank_length/2)*left_shank_frame.y)
right_thigh_center.set_pos(right_knee,(thigh_length/2)*right_thigh_frame.y)
left_thigh_center.set_pos(left_knee,(thigh_length/2)*left_thigh_frame.y)
pelvis_center.set_pos(lumbar,-1*(pelvis_length/2)*pelvis_frame.y)
torso_center.set_pos(lumbar,(torso_length/2)*torso_frame.y)


constants_dict = dict(zip(constants,numerical_constants))

foot_shape = Cylinder(radius=foot_width/2,length = constants_dict[foot_length],color='blue')
shank_shape = Cylinder(radius=0.08,length = constants_dict[shank_length],color='red')
thigh_shape = Cylinder(radius=0.08,length = constants_dict[thigh_length],color='green')
pelvis_shape = Cylinder(radius=0.08,length = constants_dict[pelvis_length],color='orange')
torso_shape = Cylinder(radius=0.08,length = constants_dict[torso_length],color='purple')


right_foot_viz_frame = VisualizationFrame('Right Foot', right_foot_frame,right_foot_center,foot_shape)
left_foot_viz_frame = VisualizationFrame('Left Foot', left_foot_frame,left_foot_center,foot_shape)
right_shank_viz_frame = VisualizationFrame('Right Shank', right_shank_frame,right_shank_center,shank_shape)
left_shank_viz_frame = VisualizationFrame('Left Shank', left_shank_frame,left_shank_center,shank_shape)
right_thigh_viz_frame = VisualizationFrame('Right Thigh',right_thigh_frame,right_thigh_center,thigh_shape)
left_thigh_viz_frame = VisualizationFrame('Left Thigh',left_thigh_frame,left_thigh_center,thigh_shape)
pelvis_viz_frame = VisualizationFrame('Pelvis',pelvis_frame,pelvis_center,pelvis_shape)
torso_viz_frame = VisualizationFrame('Torso',torso_frame,torso_center,torso_shape)

scene = Scene(world_frame,O)

scene.visualization_frames = [right_heel_viz_frame,
                              right_toe_viz_frame,
                              left_heel_viz_frame,
                              left_toe_viz_frame,
                              right_ankle_viz_frame,
                              left_ankle_viz_frame,
                              right_knee_viz_frame,
                              left_knee_viz_frame,
                              right_hip_viz_frame,
                              left_hip_viz_frame,
                              lumbar_viz_frame,
                              head_viz_frame,
                              right_foot_viz_frame,
                              left_foot_viz_frame,
                              right_shank_viz_frame,
                              left_shank_viz_frame,
                              right_thigh_viz_frame,
                              left_thigh_viz_frame,
                              pelvis_viz_frame,
                              torso_viz_frame]

scene.states_symbols = coordinates + speeds
scene.constants = constants_dict
scene.states_trajectories = y

if viz_switch:
    scene.display()



c:\Users\anbes\Documents\ReactiveBalanceSim\Phase 1 - Create Model\pydy-resources
Serving HTTP on 127.0.0.1 port 8000 ...
To view visualization, open:

http://localhost:8000/index.html?load=2025-03-11_13-46-23_scene_desc.json
Press Ctrl+C to stop server...


127.0.0.1 - - [11/Mar/2025 13:46:30] "GET /index.html?load=2025-03-11_13-46-23_scene_desc.json HTTP/1.1" 200 -
127.0.0.1 - - [11/Mar/2025 13:46:30] "GET /2025-03-11_13-46-23_scene_desc.json HTTP/1.1" 200 -
127.0.0.1 - - [11/Mar/2025 13:46:30] code 404, message File not found
127.0.0.1 - - [11/Mar/2025 13:46:30] "GET /fonts/glyphicons-halflings-regular.woff2 HTTP/1.1" 404 -
127.0.0.1 - - [11/Mar/2025 13:46:31] "GET /2025-03-11_13-46-23_simulation_data.json HTTP/1.1" 200 -
